## Creation de la database pour REVE solution 

In [788]:
import pandas as pd 
import warnings 
import math 
import re 

from library import *

warnings.filterwarnings("ignore")

In [789]:
matching_colonnes_copros = {
    "Numéro d'immatriculation":"Immatriculation_Copro",
    "Nom d’usage de la copropriété":"Nom",
    "Numéro et Voie (adresse de référence)":"Rue",
    "Code postal (adresse de référence)":"Code_Postal",
    "Commune (adresse de référence)":"Ville",
    "Code INSEE commune 1":"Code_Commune_INSEE",
    "long":"Longitude",
    "lat":"Latitude",
    "Nombre total de lots":"Nombre_Total_Lots",
    "Nombre total de lots à usage d’habitation, de bureaux ou de commerces":"Nombre_Total_Lots_HBC",
    "Nombre de lots à usage d’habitation":"Nombre_Total_Lots_Habitation",
    "Nombre de lots de stationnement":"Nombre_Total_Lots_Stationnement",
    "Résidence service":"Residence_Service",
    "Syndicat coopératif":"Syndicat_Cooperatif",
    "Syndicat principal ou syndicat secondaire":"Syndicat_Principal",
    "Si secondaire, n° d’immatriculation du principal":"Immatriculation_Syndicat_Principal",
    "Nombre d’ASL auxquelles est rattaché le syndicat de copropriétaires":"Nombre_ASL",
    "Nombre d’AFUL auxquelles est rattaché le syndicat de copropriétaires":"Nombre_AFUL",
    "Nombre d’Unions de syndicats auxquelles est rattaché le syndicat de copropriétaires":"Nombre_Union_Syndicat",
    "Référence Cadastrale 1":"Reference_Cadastrale",
    "Copro dans ACV":"Copro_Dans_ACV",
    "Copro dans PVD":"Copro_Dans_PVD",
    "Copro aidée":"Copro_Aidee",
    "EPCI":"Code_EPCI",

    "Identification du représentant légal  (raison sociale et le numéro SIRET du syndic professionnel ou Civilité/prénom/ nom du syndic bénévole ou coopératif)" : "SIRET_Syndic_Nom_Syndic",
    "Siret représentant légal (si existe)" : "SIRET_Syndic",
    "Type de syndic : bénévole / professionnel / non connu" : "Type_Syndic"
}

col_copros = [
    "Immatriculation_Copro",
    "Nom",
    "Rue",
    "Code_Postal",
    "Ville",
    "Code_Commune_INSEE",
    "Departement",
    "Longitude",
    "Latitude",
    "Nombre_Total_Lots",
    "Nombre_Total_Lots_HBC",
    "Nombre_Total_Lots_Habitation",
    "Nombre_Total_Lots_Stationnement",
    "Residence_Service",
    "Syndicat_Cooperatif",
    "Syndicat_Principal",
    "Immatriculation_Syndicat_Principal",
    "Nombre_ASL",
    "Nombre_AFUL",
    "Nombre_Union_Syndicat",
    "Debut_Construction",
    "Fin_Construction",
    "Reference_Cadastrale",
    "Copro_Dans_ACV",
    "Copro_Dans_PVD",
    "Copro_Aidee",
    "Code_EPCI",
    
    "SIRET_Syndic",
    "Type_Syndic",
    'Nom_Syndic',
]

col_syndics = [
    "SIRET", 
    "Nom", 
    
    "Categorie", 
    "Description", 
    "Rue", 
    "Ville", 
    "Code_Postal", 
    "Code_Pays", 
    "Effectif", 
    "Typologie", 
    "Forme_Juridique_INSEE", 
    "Date_Creation", 
    "Autre_Denomination", 
    "Type", 
    "Raison_Social", 
    "Code_APE", 
    "Mandat_En_Cours", 
    "Tel_1", 
    "Tel_2", 
    "Tel_3", 
    "Tel_4", 
    "Tel_5", 
    "Tel_6", 

    "URL_Linkedin"
    "URL_Website", 
    "URL_Solocal",

    "URL_Linkedin_Entreprise", 
    "ID_Linkedin_Entreprise", 
    "Statut", 
    "Record_ID_Hubspot",
]

### **RNC_Copros**

#### Clean 

In [790]:
path_copros = "data/raw/rnc_copros_france.csv"
copros_df = pd.read_csv(path_copros, dtype={'Siret représentant légal (si existe)': str})

Date

In [791]:
def date_construction(x) :
    if x =="" :
        return '0000 0000'
    elif "AVANT" in x :
        return "0000 " + x[-4:]
    elif "A_COMPTER_DE" in x :
        return x[-4:] + " 0000"
    elif "DE" in  x and "A" in x:
        return x[3:7] + " " + x[-4:]
    else :
        return "0000 0000"

copros_df['Période de construction'] = copros_df['Période de construction'].fillna("?")
copros_df['Période de construction'] =copros_df['Période de construction'].apply(lambda x : date_construction(x))

copros_df['Debut_Construction'] =copros_df['Période de construction'].apply(lambda x : x[:4])
copros_df['Fin_Construction'] =copros_df['Période de construction'].apply(lambda x : x[-4:])

Renommer colonnes

In [792]:
copros_df.rename(matching_colonnes_copros, axis=1, inplace=True)

Int -> Float

In [793]:
def int2float(colname) : 
    copros_df[colname] = pd.to_numeric(copros_df[colname], errors='coerce').fillna(-1).astype(int)
    return copros_df

copros_df = int2float('Code_Postal')
copros_df = int2float('Nombre_Total_Lots')
copros_df = int2float('Nombre_Total_Lots_Habitation')
copros_df = int2float('Nombre_Total_Lots_Stationnement')
copros_df = int2float('Nombre_Total_Lots_HBC')
copros_df = int2float('Nombre_ASL')
copros_df = int2float('Nombre_AFUL')
copros_df = int2float('Nombre_Union_Syndicat')
copros_df = int2float('Code_EPCI')

Adresse

In [794]:
# Définir le DataFrame
def expand_abbreviations(address):
    abbreviations = {
        "av": "avenue",
        "all": "allée",
        "r": "rue",
        "pl": "place",
        "bd": "boulevard",
        "imp": "impasse",
        "chem": "chemin",
        "sq": "square",
        "cte": "côte",
        "prom": "promenade",
        "espl": "esplanade",
        "qu": "quai",
        "pass": "passage",
        "crs": "cours",
        "carref": "carrefour",
        "sent": "sentier",
        "cor": "corridor",
        "pce": "place",
        "rte": "route",
        "rt": "route",
        "roc": "rocade",
    }

    if pd.notna(address):
        for abbrev, full_form in abbreviations.items():
            address = address.replace(abbrev + " ", full_form + " ")
    return address

#Rue
copros_df['Rue'] = copros_df['Rue'].astype(str)
copros_df['Rue'] = copros_df['Rue'].apply(expand_abbreviations)


# Département 
copros_df['Departement'] = copros_df['Code_Postal'].apply(lambda x : int(str(x)[:2]))

Oui/non -> bool

In [795]:
def YN2bool(x) :
    copros_df[x] = copros_df[x].map({'Oui': 1, 'Non': 0}).fillna(-1).astype(int)
    return copros_df

copros_df = YN2bool('Residence_Service')
copros_df = YN2bool('Syndicat_Cooperatif')
copros_df = YN2bool('Syndicat_Principal')
copros_df = YN2bool('Copro_Dans_ACV')
copros_df = YN2bool('Copro_Dans_PVD')
copros_df = YN2bool('Copro_Aidee')

Extraction SIRET + Nom Syndic 

In [796]:
def nom_syndic(x) : 
    if pd.isna(x) :
        return '?'
    
    x = x.split(" ")
    
    if len(x[-1]) == 14 :
        return ' '.join(x[:-1])

    else : 
        return ' '.join(x)

copros_df['Nom_Syndic'] = copros_df['SIRET_Syndic_Nom_Syndic'].apply(lambda x : nom_syndic(x))
copros_df = copros_df.drop('SIRET_Syndic_Nom_Syndic', axis=1)

Récupérer les colonnes d'intérêt

In [797]:
copros_df = copros_df[col_copros]
copros_df = copros_df.fillna('null')

In [798]:
print(len(copros_df), ' copros dans RNC')

572542  copros dans RNC


#### Graphiques copros

In [799]:
df2 = copros_df.copy()

def departement(x) : 
    x = str(x)
    return x[:2]

df2["Département"] = df2["Code_Postal"].apply(lambda x : departement(x))

In [800]:
df2_dep = df2.groupby("Département").count()
df2_dep = df2_dep.reset_index()

dep_a_garder = ["91", "92", "93", "94", "95"]
df2_dep = df2_dep.loc[df2_dep["Département"].isin(dep_a_garder)]

In [801]:
import plotly.express as px

fig = px.bar(df2_dep, x='Département', y='Nom')
fig.show()

In [802]:
import plotly.express as px

fig = px.pie(df2_dep, values='Nom', names='Département')
fig.show()

### **Syndics**

#### Filtrer copros/syndics sur SIRET

Récupérer data

In [803]:
# Syndics de france
path_syndic = "data/raw/syndics_france.csv"
s_df = pd.read_csv(path_syndic, dtype={'siret': str})

path_syndic2 = "data/raw/syndics_france2.csv"
s_df2 = pd.read_csv(path_syndic2, dtype={'siret': str})

#Pages jaunes 
path_pages_jaunes = "data/raw/pages_jaunes.csv"
pj_df = pd.read_csv(path_pages_jaunes, dtype={'siret': str})

#Concat syndics de France
s_df = pd.concat([s_df, s_df2])
print(len(s_df), " syndics en France")

3410086  syndics en France


Filtrer syndics : récupérer les syndics présents dans copros

In [804]:
#Récupérer SIRET de copros 
SIRET_copros = copros_df['SIRET_Syndic'].unique().tolist()
print(len(SIRET_copros) , " SIRETS dans copros")

#Filtrer syndics
s_df = s_df.loc[s_df['siret'].isin(SIRET_copros)]
print(len(s_df), ' syndics dans copros (match sur SIRET)')
print(3410086 - len(s_df), ' syndics non trouvés')

4466  SIRETS dans copros
4269  syndics dans copros (match sur SIRET)
3405817  syndics non trouvés


Marquer dans copros les copros qui ont un match dans syndic 

In [805]:
#Récupérer SIRET de syndics
SIRET_syndics = s_df['siret'].unique().tolist()

def tag_copros(x, SIRET_syndics) :
    SIRET_syndic = x['SIRET_Syndic'] 
    if SIRET_syndic in SIRET_syndics :  #Match
        return 1 
    if SIRET_syndic == "?" : #Non connu / Bénévole
        return -1 
    else :  #Not a match + professionnel
        return 0 
    
#Tag copros with a match  
copros_df['Match_SIRET'] = copros_df.apply(lambda x : tag_copros(x, SIRET_syndics), axis=1)
copros_df.to_csv('data/clean/copros.csv', index=False)

In [806]:
len(copros_df.dtypes)

31

#### Clean Syndics

In [807]:
to_keep_syndics = [
            'siret',
            'dateCreationEtablissement',
            'trancheEffectifsEtablissement',
            'etablissementSiege',
            'complementAdresseEtablissement',
            'numeroVoieEtablissement', 
            'indiceRepetitionEtablissement',
            'typeVoieEtablissement', 
            'libelleVoieEtablissement',
            'codePostalEtablissement', 
            'libelleCommuneEtablissement',
            'codeCommuneEtablissement',
            'codePaysEtrangerEtablissement',
            'activitePrincipaleEtablissement',
           ]

matching_colonnes_syndics = {
            'siret' : "SIRET",
            'dateCreationEtablissement' : 'Date_Creation',
            'trancheEffectifsEtablissement' : 'Effectif',    
            'etablissementSiege' : 'Typologie',            #1 -> Siège / 0 -> Etablissement secondaire
            'complementAdresseEtablissement' : 'comp',
            'numeroVoieEtablissement' : 'num', 
            'indiceRepetitionEtablissement' : 'rep',
            'typeVoieEtablissement' : 'type', 
            'libelleVoieEtablissement' : 'voie',
            'codePostalEtablissement' : 'Code_Postal', 
            'libelleCommuneEtablissement' : 'Ville',
            'codeCommuneEtablissement' : 'Code_Commune',
            'codePaysEtrangerEtablissement' : 'Code_Pays',
            'activitePrincipaleEtablissement' : 'Code_APE',
}

Ne garder que les colonnes d'intérêt + renommer 

In [808]:
s_df = s_df[to_keep_syndics]
s_df.rename(matching_colonnes_syndics, axis=1, inplace=True)

Adresse

In [809]:
# Définir le DataFrame
def expand_abbreviations(address):
    abbreviations = {
        "av": "avenue",
        "all": "allée",
        "pl": "place",
        "bd": "boulevard",
        "imp": "impasse",
        "chem": "chemin",
        "sq": "square",
        "cte": "côte",
        "prom": "promenade",
        "espl": "esplanade",
        "qu": "quai",
        "pass": "passage",
        "crs": "cours",
        "carref": "carrefour",
        "sent": "sentier",
        "cor": "corridor",
        "pce": "place",
        "rte": "route",
        "rt": "route",
        "roc": "rocade",
    }

    address = str(address).lower()
    if pd.notna(address):
        for abbrev, full_form in abbreviations.items():
            address = address.replace(abbrev, full_form)
    return address.upper()

def adresse(x) : 
    num, rep, type, voie, comp = x['num'], x['rep'], x['type'], x['voie'], x['comp']
    adresse = ""
    for i in [comp, num, rep, type, voie] :
        if isinstance(i, float) and math.isnan(i) :
            i = ""
        else : 
            adresse += i 
            adresse += " "

    return adresse

s_df['type'] = s_df['type'].apply(expand_abbreviations)
s_df['Adresse'] = s_df.apply(lambda x : adresse(x), axis=1)

Siège

In [810]:
def siege(x) :
    if x == 1 :
        return "Siège"
    else :
        return "Etablissement secondaire"

s_df['Typologie'] = s_df['Typologie'].apply(lambda x : siege(x))

Code PAYS

In [811]:
s_df['Code_Pays'] = s_df['Code_Pays'].fillna('FR')

Departement

In [812]:
s_df["Departement"] = s_df["Code_Postal"].apply(lambda x : departement(x))
s_df["Departement"] = s_df["Departement"].apply(lambda x : 'None' if x == '[N' else x)

Nom Syndic 

In [813]:
nom_syndics = copros_df[['Nom_Syndic', 'SIRET_Syndic']]
nom_syndics = nom_syndics.drop_duplicates()
s_df = s_df.merge(nom_syndics, left_on='SIRET', right_on='SIRET_Syndic', how='left')

Derniers changements + Sauvegarde

In [814]:
#s_df = s_df.drop(['comp', 'num', 'rep', 'type', 'voie', 'SIRET_Syndic'], axis=1)
s_df.fillna('null', inplace=True)
s_df = s_df[['SIRET', 'Nom_Syndic', 
            'Adresse', 'Code_Postal', 'Ville', 'Code_Commune', 'Code_Pays', 'Departement',
            'Code_APE', 'Date_Creation', 'Effectif', 'Typologie']]
s_df.to_csv('data/clean/syndics.csv', index=False)

In [815]:
s_df

,SIRET,Nom_Syndic,Adresse,Code_Postal,Ville,Code_Commune,Code_Pays,Departement,Code_APE,Date_Creation,Effectif,Typologie
0,27240001100032,OFFICE PUBLIC DE L'HABITAT PERIGORD HABITAT,CRE@VALLEE NORD - CREAPARK BAT 2 212 BOULEVARD...,24660,COULOUNIEIX-CHAMIERS,24138,FR,24,68.20B,2015-06-15,22,Siège
1,30003136600051,CABINET TAPISSIER & ASSOCIES,1 AVENUE DE CONTADES,49000,ANGERS,49007,FR,49,68.31Z,2023-03-13,null,Siège
2,30007257600014,AGENCE SECKLER,9 AVENUE FOCH,68000,COLMAR,68066,FR,68,68.31Z,1974-01-01,11,Siège
3,30016560200024,LOCATRIM CASSIS,2 RUE ADOLPHE THIERS,13260,CASSIS,13022,FR,13,68.31Z,1989-04-10,02,Siège
4,30017936300050,REGIE MIALON,47 - 49 47 AVENUE DES ETATS UNIS,63000,CLERMONT-FERRAND,63113,FR,63,68.31Z,2009-11-15,12,Siège
...,...,...,...,...,...,...,...,...,...,...,...,...
4264,48127755600017,SIMARD*/ISABELLE /,56 AVENUE DE LA REPUBLIQUE,71210,MONTCHANIN,71310,FR,71,69.10Z,2005-02-07,NN,Siège
4265,47889558400025,SELARL LEGRAND ET ANTONIOTTI,BAT B RN 197 RES LES DOMAINES DE L'ILE ROUSSE,20220,L'ILE ROUSSE,2B134,FR,20,71.12A,2013-06-25,02,Siège
4266,50392965500028,JG EXPERTISE IMMOBILIER PATRIMOINE FINANCE,IMMEUBLE LE MERCURE BATIMENT C 48 CHE DE L HOM...,30900,NIMES,30189,FR,30,69.10Z,2014-04-07,NN,Siège
4267,39371821800018,SCP JUNQUA LAMARQUE & ASSOCIES,CENTRE JORLIS-ALLIANCE 3 RUE DU PONT DE L AVEU...,64600,ANGLET,64024,FR,64,69.10Z,1994-01-01,02,Siège


#### Pages Jaunes

In [816]:
pj_df = pd.read_csv(path_pages_jaunes, dtype={'siret': str})

SIRET/Code NAF/ Effectic / Typologie

In [817]:
def extract_SIRET(x) :
    x = x.replace(':', ',').replace('{', '').replace('}', '').replace('\'', '').replace('\'', '').replace('"', "").split(',')
    if x[0] == 'SIRET' :
        return x[1].replace(' ', '')

def extract_NAF(x) :
    x = x.replace(':', ',').replace('{', '').replace('}', '').replace('\'', '').replace('\'', '').replace('"', "").split(',')
    if len(x) >= 4 and x[2] == ' Code NAF' :
        return x[3].replace(' ', '')
    
def extract_effectif(x) :
    x = x.replace(':', ',').replace('{', '').replace('}', '').replace('\'', '').replace('\'', '').replace('"', "").split(',')
    if len(x) >= 4 and x[4] == ' Effectif de létablissement' :
        return x[5]

def extract_typologie(x) : 
    x = x.replace(':', ',').replace('{', '').replace('}', '').replace('\'', '').replace('\'', '').replace('"', "").split(',')
    if len(x) >= 8 and x[6] == ' Typologie de létablissement' :
        return x[7]
    elif len(x) >= 6 and x[4] == ' Typologie de létablissement' :
        return x[5]

pj_df['SIRET'] = pj_df["get_établissement"].apply(extract_SIRET)
pj_df['Code_APE'] = pj_df["get_établissement"].apply(extract_NAF)
pj_df['Effectif'] = pj_df["get_établissement"].apply(extract_effectif)
pj_df['Typologie'] = pj_df["get_établissement"].apply(extract_typologie)

pj_df.drop(['get_établissement'], axis=1, inplace=True)

Adresse

In [818]:
pj_df[["Adresse", "Code_Postal", "Ville"]] = pj_df["location"].apply(extract_zipcode_city).apply(pd.Series)
pj_df.drop("location", axis=1, inplace=True)
pj_df["Département"] = pj_df["Code_Postal"].apply(lambda x : x[:2] if x is not None else None)
pj_df['Code_Pays'] = 'FR'

pj_df.drop("contractor_link", axis=1, inplace=True)

Horaires

In [819]:
#Extract opening and closing hours 
days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]

horaires_df = pj_df[["SIRET", "information"]]

horaires_df[days] = horaires_df["information"].apply(extract_information).apply(pd.Series)

#Transform time strings to time objects
for day in days : 
    horaires_df[day] = horaires_df[day].apply(str2hours)

horaires_df = horaires_df.drop("information", axis=1)

to_keep = ["SIRET"]

for j in ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi'] :
    debut = j + ' Début'
    fin = j + ' Fin'
    to_keep.append(debut)
    to_keep.append(fin)

    horaires_df[debut] = horaires_df[j].apply(lambda x : x[0])
    horaires_df[fin] = horaires_df[j].apply(lambda x : x[1])

horaires_df = horaires_df.fillna('fermé')

horaires_df = horaires_df[to_keep]

new_df = pd.DataFrame(columns=["SIRET_Syndic", "Jour_Semaine", "Heure_Ouverture", "Heure_Fermeture"])

for index, row in horaires_df.iterrows() :
    SIRET = row["SIRET"]
    for j in ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi'] :
        debut = j + ' Début'
        fin = j + ' Fin'
        new_df = new_df.append({"SIRET_Syndic" : SIRET, "Jour_Semaine" : j, "Heure_Ouverture" : row[debut], "Heure_Fermeture" : row[fin]}, ignore_index=True)

new_df.to_csv("data/clean/horaires_syndics.csv", index=False)

pj_df = pj_df.drop('information', axis=1)

Présentation 

In [820]:
def clean_presentation(x) :
    x = x.replace('[', '').replace(']', '').split(',')
    new_x = []

    for i in x :  
        cpt_deb = 0 
        cond_deb = True
        while cond_deb : 
            if len(i) > 0 and (i[0] == " " or i[0] == "'" or i[0] == '"'):
                i = i[1:]
            else : 
                cond_deb = False
            cpt_deb += 1
        cpt_end = 0 
        cond_end = True
        while cond_end : 
            if len(i) > 0 and (i[-1] == " " or i[-1] == "'" or i[-1] == '"'):
                i = i[:-1]
            else : 
                cond_end = False
            cpt_end += 1
        new_x.append(i.lower())

    return new_x

#Clean presentation
df_tags = pj_df[['SIRET', 'presentation']]
df_tags["presentation"] = df_tags["presentation"].apply(lambda x : clean_presentation(x))
df_tags = df_tags.explode("presentation")

#Creation de la table des tags / ID_tags 
tags = df_tags["presentation"].unique().tolist()
tags = [x for x in tags if x != '']
tags = list(set(tags))

tags_id_tags = pd.DataFrame(columns=["Service"])
tags_id_tags["Service"] = tags
tags_id_tags['ID_Service'] = tags_id_tags.index

tags_id_tags.to_csv("data/clean/tags.csv")

#Creation de la table des tags / ID_syndics
tags_id_tags_dict = dict(zip(tags_id_tags["Service"], tags_id_tags["ID_Service"]))
df_tags.drop(df_tags[df_tags['presentation'] == ''].index, inplace=True)
df_tags['presentation'] = df_tags['presentation'].map(tags_id_tags_dict)
df_tags.columns = ['SIRET_Syndic', 'ID_Service']

df_tags.to_csv("data/clean/tags_syndics.csv", index=False)

pj_df = pj_df.drop('presentation', axis=1)

Forme juridique / Création d'entreprise / Effectif / Autres dénominations

In [821]:
# Fonction pour extraire la valeur d'une clé en toute sécurité
def extract_key(data, key):
    try:
        return eval(data)[key]
    except (KeyError, TypeError):
        return None

# Fonction pour convertir la date en format correct
def convert_date(x):
    try:
        x = x.split(" ")
        jour = x[0]
        if len(jour) == 1 :
            jour = "0" + jour

        mois = x[1] #janvier
        annee = x[2]

        dict_mois = {"janvier": "01",
        "février": "02",
        "mars": "03",
        "avril": "04",
        "mai": "05",
        "juin": "06",
        "juillet": "07",
        "août": "08",
        "septembre": "09",
        "octobre": "10",
        "novembre": "11",
        "décembre": "12"}

        mois_numero = dict_mois[mois]

        date = annee + "-" + mois_numero + "-" + jour
        return date
    except :
        return None
        
# Fonction pour extraire le maximum de la fourchette d'effectif
def extract_effectif(effectif_str):
    try:
        if effectif_str is not None:
            return max(map(str, effectif_str.split(' à ')))
        else:
            return None
    except (ValueError, TypeError):
        return None
    
# Fonction pour séparer les numéros de téléphone
def separer_numeros(telephones):
    telephones = telephones.strip("[]")  # Supprimer les crochets
    numeros_separes = telephones.replace("'", "").replace(" ", "").split(',')
    # Remplir les colonnes Tel_1 à Tel_6 avec les numéros séparés
    result = {}
    for i in range(6):
        if i < len(numeros_separes):
            result[f'Tel_{i+1}'] = numeros_separes[i]
        else:
            result[f'Tel_{i+1}'] = None
    return result

pj_df["Effectif"] = pj_df['get_entreprise'].apply(lambda x: extract_effectif(extract_key(x, "Effectif de l'entreprise")))
pj_df["Date_Creation"] = pj_df['get_entreprise'].apply(lambda x: convert_date(extract_key(x, "Création d'entreprise")))
pj_df['Autre_Denomination'] = pj_df['get_entreprise'].apply(lambda x: extract_key(x, 'Autres dénominations'))
pj_df['Forme_Juridique_INSEE'] = pj_df['get_entreprise'].apply(lambda x: extract_key(x, 'Forme juridique'))

pj_df = pj_df.drop("get_entreprise", axis=1)

Sites

In [822]:
def sites(x) : 
    x = x.replace("[", "").replace("]", "").replace("'", "").replace('"', "").split(",")
    new_x = []
    for i in x : 
        if i[0] == " " :
            i = i[1:]
        if i != 'None' : 
            new_x.append(i)
        else : 
            new_x.append('')
    return new_x[0], new_x[1]

pj_df['URL_Website'], pj_df['URL_Solocal']= zip(*pj_df['sites'].map(sites))
pj_df = pj_df.drop(['sites', 'link'], axis=1)

pj_df.rename(columns={"url" : "URL_PagesJaunes"}, inplace=True)

Téléphone

In [823]:
def telephone(x) : 
    x = x.replace("[", "").replace("]", "").replace("'", "").replace('"', "").replace(' ', '').split(",")
    x = ' '.join(x)
    return x 

pj_df['Telephones'] = pj_df['contractor_phone'].apply(telephone)
pj_df = pj_df.drop(['contractor_phone', 'contact'], axis=1)

Linkedin

In [824]:
pj_df['URL_Linkedin_Entreprise'] = ''
pj_df['ID_Linkedin_Entreprise'] = ''

Rename and reorder

In [825]:
pj_df = pj_df.rename(columns={'title': 'Nom', 
                              'sub_title': 'Catégorie', 
                              'description' : 'Description', 
                              'Département' : 'Departement'}) 

col_pj = [
    "SIRET",
    "Nom",
    "Autre_Denomination",

    "Catégorie",
    "Description",

    "Adresse",
    "Code_Postal",
    "Ville",
    "Departement",
    "Code_Pays",

    "Telephones",

    "Effectif",
    "Date_Creation",
    "Forme_Juridique_INSEE",
    "Code_APE",
    "Typologie",

    "URL_PagesJaunes",
    "URL_Website",
    "URL_Solocal",

    "URL_Linkedin_Entreprise",
    "ID_Linkedin_Entreprise",
]

pj_df = pj_df[col_pj]


Filter

In [826]:
start = len(pj_df)
print(start)

#Only keep Rows where SIRET in copros 
pj_df = pj_df.loc[pj_df['SIRET'].isin(SIRET_copros)]
pj_df.to_csv('data/clean/pages_jaunes.csv', index=False)

print(len(pj_df), start - len(pj_df))

1403
228 1175


In [827]:
SIRET_copros = copros_df['SIRET_Syndic'].unique().tolist()
SIRET_pj = pj_df['SIRET'].unique().tolist()
SIRET_syndics = s_df['SIRET'].unique().tolist()

print(f"RNC : {len(SIRET_copros)},  Syndics : {len(SIRET_syndics)}, Pj : {len(SIRET_pj)}")

print("En commun syndics + PJ : ", len(set(SIRET_syndics).intersection(set(SIRET_pj))))

print('Nombre total de syndics trouvés : ', len(set(SIRET_copros).intersection(set(SIRET_syndics).union(set(SIRET_pj)))), '/', len(SIRET_copros))

print("Il en manque : ", len(SIRET_copros) - len(set(SIRET_copros).intersection(set(SIRET_syndics).union(set(SIRET_pj)))))

RNC : 4466,  Syndics : 4269, Pj : 228
En commun syndics + PJ :  225
Nombre total de syndics trouvés :  4272 / 4466
Il en manque :  194


### Merge syndics + Pages jaunes

In [828]:
syndics = pd.read_csv("data/clean/syndics.csv", dtype={'SIRET': str})
pj = pd.read_csv("data/clean/pages_jaunes.csv", dtype={'SIRET': str})


syndics.head(2)

,SIRET,Nom_Syndic,Adresse,Code_Postal,Ville,Code_Commune,Code_Pays,Departement,Code_APE,Date_Creation,Effectif,Typologie
0,27240001100032,OFFICE PUBLIC DE L'HABITAT PERIGORD HABITAT,CRE@VALLEE NORD - CREAPARK BAT 2 212 BOULEVARD...,24660,COULOUNIEIX-CHAMIERS,24138,FR,24,68.20B,2015-06-15,22,Siège
1,30003136600051,CABINET TAPISSIER & ASSOCIES,1 AVENUE DE CONTADES,49000,ANGERS,49007,FR,49,68.31Z,2023-03-13,NaN,Siège


In [829]:
pj.head(2)

,SIRET,Nom,Autre_Denomination,Catégorie,Description,Adresse,Code_Postal,Ville,Departement,Code_Pays,...,Effectif,Date_Creation,Forme_Juridique_INSEE,Code_APE,Typologie,URL_PagesJaunes,URL_Website,URL_Solocal,URL_Linkedin_Entreprise,ID_Linkedin_Entreprise
0,31180020500024,C.G.I Le Goff,Le Goff Bernard Cabinet CABINET GEST IMMOB B Y...,Syndics de copropriétés,Notre cabinet vous propose son expertise dans ...,5 boulevard Edgar Quinet,92700,Colombes,92,FR,...,19 salariés,1980-01-01,5710.0,6832A,Siège,https://www.pagesjaunes.fr/pros/00091222,www.cgilegoff.fr,http://cabinetgestimmobbylegoff.site-solocal.com,NaN,NaN
1,49430420700025,Excellence Gestion,NaN,Administrateurs de biens,Notre société met à votre disposition une équi...,15 rue Chante Coq,92800,Puteaux,92,FR,...,5 salariés,2007-01-22,5710.0,6832A,Siège,https://www.pagesjaunes.fr/pros/53322784,www.excellencegestion.fr,NaN,NaN,NaN


In [830]:
merged_df = pd.merge(syndics, pj, on='SIRET', suffixes=('_s', '_pj'), how='outer')

Check colonnes en commun 

In [831]:
def choose_nom(x) :
    if pd.isna(x['Nom_Syndic']) :
        return x['Nom']
    elif pd.isna(x['Nom']) :       
        return x['Nom_Syndic']
    else :
        return x['Nom_Syndic'] + " (pj : " +  x['Nom'] + ")"


merged_df['New_Nom'] = merged_df.apply(lambda x : choose_nom(x), axis=1)
merged_df = merged_df.drop(['Nom_Syndic', 'Nom'], axis=1)
merged_df.rename(columns={'New_Nom' : 'Nom'}, inplace=True)

In [832]:
def choose_adresse(x) :
    if pd.isna(x['Adresse_s']) :
        return x['Adresse_pj']
    elif pd.isna(x['Adresse_pj']) :       
        return x['Adresse_s']
    else :
        return x['Adresse_s'] + " (pj : " +  x['Adresse_pj'] + ")"

merged_df['Adresse'] = merged_df.apply(lambda x : choose_adresse(x), axis=1)
merged_df = merged_df.drop(['Adresse_s', 'Adresse_pj'], axis=1)

In [833]:
def choose_CodePostal(x):
    if pd.isna(x['Code_Postal_s']):
        cp = str(x['Code_Postal_pj'])
        if cp[-2] == '.' : 
            cp = cp[:-2]
        return cp
    elif pd.isna(x['Code_Postal_pj']):
        if x['Code_Postal_s'] != '[ND]':  # Check if the value is not 'ND'
            cp = str(x['Code_Postal_s'])
            if cp[-2] == '.' : 
                cp = cp[:-2]
            return cp
        else:
            return None  # or another placeholder value
    else:
        cp = str(x['Code_Postal_pj'])
        if cp[-2] == '.' : 
            cp = cp[:-2]
        return cp

merged_df['Code_Postal'] = merged_df.apply(choose_CodePostal, axis=1)
merged_df = merged_df.drop(['Code_Postal_s', 'Code_Postal_pj'], axis=1)

In [834]:
def choose_Ville(x) :
    if pd.isna(x['Ville_s']) :
        return x['Ville_pj']
    elif pd.isna(x['Ville_pj']) :       
        return x['Ville_s']
    else :
        return x['Ville_s'] 

merged_df['Ville'] = merged_df.apply(lambda x : choose_Ville(x), axis=1)
merged_df = merged_df.drop(['Ville_s', 'Ville_pj'], axis=1)

In [835]:
def choose_CodePays(x) :
    if pd.isna(x['Code_Pays_s']) :
        return x['Code_Pays_pj']
    elif pd.isna(x['Code_Pays_pj']) :       
        return x['Code_Pays_s']
    else :
        return x['Code_Pays_s']

merged_df['Code_Pays'] = merged_df.apply(lambda x : choose_CodePays(x), axis=1)
merged_df = merged_df.drop(['Code_Pays_s', 'Code_Pays_pj'], axis=1)

In [836]:
def choose_Departement(x) :
    if pd.isna(x['Departement_s']) :
        return str(int(x['Departement_pj']))
    elif pd.isna(x['Departement_pj']) :       
        return x['Departement_s']
    else :
        return x['Departement_s']

merged_df['Departement'] = merged_df.apply(lambda x : choose_Departement(x), axis=1)

In [837]:
def choose_APE(x) :
    if pd.isna(x['Code_APE_s']) :
        return x['Code_APE_pj']
    elif pd.isna(x['Code_APE_pj']) :       
        return x['Code_APE_s'].replace('.','')
    else :
        return x['Code_APE_pj']

merged_df['Code_APE'] = merged_df.apply(lambda x : choose_APE(x), axis=1)
merged_df = merged_df.drop(['Code_APE_s', 'Code_APE_pj'], axis=1)

In [838]:
def choose_DateCreation(x) :
    if pd.isna(x['Date_Creation_s']) :
        return x['Date_Creation_pj']
    elif pd.isna(x['Date_Creation_pj']) :       
        return x['Date_Creation_s']
    else :
        return x['Date_Creation_s']

merged_df['Date_Creation'] = merged_df.apply(lambda x : choose_DateCreation(x), axis=1)
merged_df = merged_df.drop(['Date_Creation_s', 'Date_Creation_pj'], axis=1)

In [839]:
def choose_effectif(x) :
    if pd.isna(x['Effectif_s']) and pd.isna(x['Effectif_pj']) :
        return '?'
    elif pd.isna(x['Effectif_pj']) :       
        return x['Effectif_s']
    elif pd.isna(x['Effectif_s']) :
        return 'None'
    else :
        return x['Effectif_s'] + " (pj : " +  x['Effectif_pj'].split(" ")[1] + ")"

merged_df['Effectif'] = merged_df.apply(lambda x : choose_effectif(x), axis=1)
merged_df = merged_df.drop(['Effectif_s', 'Effectif_pj'], axis=1)

In [840]:
def choose_typologie(x) :
    if pd.isna(x['Typologie_s']) :
        return x['Typologie_pj']
    elif pd.isna(x['Typologie_pj']) :       
        return x['Typologie_s']
    else :
        return x['Typologie_s']

merged_df['Typologie'] = merged_df.apply(lambda x : choose_typologie(x), axis=1)
merged_df = merged_df.drop(['Typologie_s', 'Typologie_pj'], axis=1)

Check types colonnes 

In [841]:
merged_df["Forme_Juridique_INSEE"] = merged_df["Forme_Juridique_INSEE"].apply(lambda x : x if not pd.isna(x) else 0)
merged_df["Forme_Juridique_INSEE"] = merged_df["Forme_Juridique_INSEE"].astype(int)

In [842]:
merged_df["URL_Linkedin_Entreprise"] = merged_df["URL_Linkedin_Entreprise"].apply(lambda x : str(x) if not pd.isna(x) else '?')
merged_df["ID_Linkedin_Entreprise"] = merged_df["URL_Linkedin_Entreprise"].apply(lambda x : str(x) if not pd.isna(x) else '?')


In [843]:
col_merged = ['SIRET', 
              'Nom',
              'Autre_Denomination',
              'Catégorie',
              'Description',
              
              'Adresse',
              'Code_Postal', 
              'Ville',  
              'Code_Commune',
              'Code_Pays',  
              'Departement',

              'Telephones',
              'URL_PagesJaunes', 
              'URL_Website', 
              'URL_Solocal',
              'URL_Linkedin_Entreprise', 
              'ID_Linkedin_Entreprise',
              
              'Forme_Juridique_INSEE',
              'Code_APE', 
              'Date_Creation',
              'Effectif', 
              'Typologie'
]

In [844]:
merged_df = merged_df[col_merged]
merged_df = merged_df.fillna('null')
merged_df.to_csv('data/clean/merged_syndics.csv', index=False)
merged_df

,SIRET,Nom,Autre_Denomination,Catégorie,Description,Adresse,Code_Postal,Ville,Code_Commune,Code_Pays,...,URL_PagesJaunes,URL_Website,URL_Solocal,URL_Linkedin_Entreprise,ID_Linkedin_Entreprise,Forme_Juridique_INSEE,Code_APE,Date_Creation,Effectif,Typologie
0,27240001100032,OFFICE PUBLIC DE L'HABITAT PERIGORD HABITAT,null,null,null,CRE@VALLEE NORD - CREAPARK BAT 2 212 BOULEVARD...,24660,COULOUNIEIX-CHAMIERS,24138,FR,...,null,null,null,?,?,0,6820B,2015-06-15,22,Siège
1,30003136600051,CABINET TAPISSIER & ASSOCIES,null,null,null,1 AVENUE DE CONTADES,49000,ANGERS,49007,FR,...,null,null,null,?,?,0,6831Z,2023-03-13,?,Siège
2,30007257600014,AGENCE SECKLER,null,null,null,9 AVENUE FOCH,68000,COLMAR,68066,FR,...,null,null,null,?,?,0,6831Z,1974-01-01,11,Siège
3,30016560200024,LOCATRIM CASSIS,null,null,null,2 RUE ADOLPHE THIERS,13260,CASSIS,13022,FR,...,null,null,null,?,?,0,6831Z,1989-04-10,02,Siège
4,30017936300050,REGIE MIALON,null,null,null,47 - 49 47 AVENUE DES ETATS UNIS,63000,CLERMONT-FERRAND,63113,FR,...,null,null,null,?,?,0,6831Z,2009-11-15,12,Siège
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,39371821800018,SCP JUNQUA LAMARQUE & ASSOCIES,null,null,null,CENTRE JORLIS-ALLIANCE 3 RUE DU PONT DE L AVEU...,64600,ANGLET,64024,FR,...,null,null,null,?,?,0,6910Z,1994-01-01,02,Siège
4268,39381579000034,CABINET CLAUDE ANDRES,null,null,null,-BATIMENT LE GALION- 1 RUE DE PULLY,67210,OBERNAI,67348,FR,...,null,null,null,?,?,0,7112A,2016-05-16,11,Siège
4269,78950592200029,GRW IMMOBILIER,null,Syndics de copropriétés,Notre agence prend en charge la gestion de vos...,56 rue Gén de Gaulle,78120,Rambouillet,null,FR,...,https://www.pagesjaunes.fr/pros/55179761,www.adcrambouillet.com,null,?,?,5710,6420Z,2012-12-15,None,Siège
4270,62980041800036,S.E.M.I.V Sté d'Economie Mixte Immobilière de ...,S.E.M.I.V SOC ECONOMIE MIXTE IMMOBILIERE VELIZY,Syndics de copropriétés,Si vous cherchez un expert en gestion de copro...,54 avenue Europe Cs,10015,78140 Vélizy Villacoublay,null,FR,...,https://www.pagesjaunes.fr/pros/05676299,www.semiv-velizy.fr,null,?,?,5515,6820A,1990-04-20,None,Siège


In [845]:
merged_df['Departement'].unique()

array(['24', '49', '68', '13', '63', '59', '31', '69', '38', '64', '71',
       '87', '75', '94', '22', '57', '83', '80', '06', '91', '21', '95',
       '67', '35', '78', '60', '51', '74', '85', '05', '42', '14', '81',
       '92', '03', '44', '29', '37', '33', '45', '16', '43', '46', '54',
       '48', '86', '97', '56', '40', '04', '30', '73', '66', '82', '20',
       '34', '76', '15', '25', '52', '01', '93', '02', '58', '77', '84',
       '50', '10', '65', '11', '47', '12', '17', '28', '39', '61', '26',
       '36', '53', '07', '90', '89', '09', '18', '23', '62', '79', '88',
       '72', '19', '41', '70', '27', 'None', '08', '32', '55'],
      dtype=object)

In [846]:
merged_df.dtypes

SIRET                      object
Nom                        object
Autre_Denomination         object
Catégorie                  object
Description                object
Adresse                    object
Code_Postal                object
Ville                      object
Code_Commune               object
Code_Pays                  object
Departement                object
Telephones                 object
URL_PagesJaunes            object
URL_Website                object
URL_Solocal                object
URL_Linkedin_Entreprise    object
ID_Linkedin_Entreprise     object
Forme_Juridique_INSEE       int32
Code_APE                   object
Date_Creation              object
Effectif                   object
Typologie                  object
dtype: object

Foreign key problem

In [847]:
merged_df.shape

(4272, 22)

In [849]:
#Dans Syndics, ajouter une ligne avec 01234567890123
merged_df.loc[len(merged_df)] = ['01234567890123'] + ['null'] * 21
merged_df.drop_duplicates(subset=['SIRET'], keep='first', inplace=True)
merged_df.to_csv('data/clean/merged_syndics.csv', index=False)

In [851]:
#Dans Copros, pour toutes les copros qui n'ont pas de match (Match_SIRET = -1, 0), 
#Remplacer le SIRET par 01234567890123
copros_df['SIRET_Syndic'] = copros_df.apply(lambda x : '01234567890123' if x['Match_SIRET'] in [-1,0] else x['SIRET_Syndic'], axis=1)

copros_df.to_csv('data/clean/copros.csv', index=False)